<img src="pr1.png" />

In [1]:
import cvxpy
import gurobipy
from cvxpy import *
import numpy as np

In [10]:
np.random.seed(42)

# initial matrix
m = np.array([[ 5 , 5 , 10, 10, 10 ],
              [ 5 , 5 , 10, 20, 10 ],
              [ 0 , 5 , 5 , 10,  5 ],
              [ 0 , 0 , 0 , 5 ,  0 ]])

N, M = m.shape
C = N * M

# delta matrix
d = np.ones((N, M)) * 6
d = d - m

def cost(i1, j1, i2, j2):
    assert(i1 < N and i2 < N and j1 < M and j2 < M and 
           i1 >= 0 and i2 >= 0 and j1 >= 0 and j2 >= 0)
    
    if i1 == i2 and j1 == j2:
        return 1000000
    
    return np.sqrt(np.power(np.abs(i1 - i2), 2) + np.power(np.abs(j1 - j2), 2))

m = m.reshape(C,1)
d = d.reshape(C,1)

f = Variable(C * C)
c = np.zeros(C * C)

for i1 in xrange(N):
    for j1 in xrange(M):
        for i2 in xrange(N):
            for j2 in xrange(M):
                c[(i1 * M + j1) * C + (i2 * M + j2)] = cost(i1, j1, i2, j2)
                
# each cell can either be only a consumer or "donor".
# otherwise we might get non integer solution
def pred(x):
    if x < 0:
        return 1
    
    return 0

flows_are_positive = [ (f[i] >= 0) for i in xrange(C * C) ]

pd = np.vectorize(pred)
is_donor = pd(d).reshape(C,)
flow_network_constraints = [ (-is_donor[i] * cvxpy.sum_entries(f[i*C:i*C + C]) + \
                             (1 - is_donor[i]) * cvxpy.sum_entries(f[i:C*C:C]) == d[i]) \
                            for i in xrange(C) ]

fnc_not_donors = [ (1 - is_donor[i]) * cvxpy.sum_entries(f[i*C:i*C + C]) == 0 for i in xrange(C) ]
fnc_donors = [ is_donor[i] * cvxpy.sum_entries(f[i:C*C:C]) == 0 for i in xrange(C) ]

obj = Minimize(c * f)

constraints = flows_are_positive + flow_network_constraints + fnc_not_donors + fnc_donors

prob = Problem(obj, constraints)
res = prob.solve(feastol=1e-8) # verbose=True, 
print res

# print zip(f.value[np.where( f.value > 0.00001 )], np.where( f.value > 0.00001 ))
# print cvxpy.sum_entries(f[13:C*C:C]).value

95.4910638676
[(matrix([[ 0.99999998,  0.99999998,  0.99999997,  0.99999993,  3.99999984,
          0.48566975,  0.99999983,  2.51433027,  2.00000003,  0.99999993,
          1.        ,  0.99999996,  4.99999992,  5.99999995,  2.00000011,
          0.51433024,  3.48566972,  3.99999983]]), array([ 40,  41,  45,  46,  70,  94,  98,  99, 150, 151, 155, 172, 175,
       176, 177, 194, 199, 277]))]
5.67797226463e-13


I've tried several similar min cost max flow models. All of them gave the same result.
As you can see some flows are not intergers. Nevertheless solution is optimal.

<img src="pr2.png" />

<img src="pr3.png" />